In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
def requester(number):
    cookies = {
        'ASP.NET_SessionId': 't2kwihwgqh1bxprzk0c31ihh',
        'BIGipServerpool_app_web_ext': '1510103562.0.0000',
    }

    headers = {
        'Connection': 'keep-alive',
        'Cache-Control': 'max-age=0',
        'Origin': 'https://consultapublica.antt.gov.br',
        'Upgrade-Insecure-Requests': '1',
        'Content-Type': 'application/x-www-form-urlencoded',
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
        'Referer': 'https://consultapublica.antt.gov.br/Site/ConsultaRNTRC.aspx/consultapublica',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7',
    }
    
    # Após analisar algumas requisições, percebi que o padrão de mudança de página era na parte a baixo
    EVENTTARGET = 'ctl00$Corpo$ucPaginatorConsultaPesquisa$ucPaginatorConsultaPesquisa_rptPages$ctl{}$lbPage'.format(number)
    
    data = {
      '__EVENTTARGET': EVENTTARGET,
      '__EVENTARGUMENT': '',
      '__LASTFOCUS': '',
      '__VIEWSTATE': '/wEPDwULLTEzMjI3Njk3NjQPFgQeGUNvZE11bmljaXBpb1RyYXNucG9ydGFkb3JlHg1yZXN1bENvbnN1bHRhZBYCZg9kFgICAw9kFgYCBQ8WAh4JaW5uZXJodG1sBRNWZXJzJiMyMjc7byAxLjAuMS4xZAIHDzwrAA0CAA8WAh4LXyFEYXRhQm91bmRnZAwUKwADBQcwOjAsMDoxFCsAAhYQHgRUZXh0BQVSTlRSQx4FVmFsdWUFBVJOVFJDHgtOYXZpZ2F0ZVVybAUYL1NpdGUvQ29uc3VsdGFSTlRSQy5hc3B4HgdUb29sVGlwBTtDb25zdWx0YSBQw7pibGljYSBkZSBUcmFuc3BvcnRhZG9yZXMgUm9kb3Zpw6FyaW9zIGRlIENhcmdhcx4HRW5hYmxlZGceClNlbGVjdGFibGVnHghEYXRhUGF0aAUYL3NpdGUvY29uc3VsdGFybnRyYy5hc3B4HglEYXRhQm91bmRnZBQrAAIWEB8EBQpQRUYgLyBDSU9UHwUFClBFRiAvIENJT1QfBgUXL1NpdGUvQ29uc3VsdGFDSU9ULmFzcHgfBwUZQ29uc3VsdGEgUMO6YmxpY2EgZG8gQ0lPVB8IZx8JZx8KBRcvc2l0ZS9jb25zdWx0YWNpb3QuYXNweB8LZ2RkAg0PZBYCZg9kFgJmD2QWAgIBD2QWCgIDDxAPFgYeDkRhdGFWYWx1ZUZpZWxkBQNrZXkeDURhdGFUZXh0RmllbGQFBXZhbHVlHwNnZBAVAxFQb3IgVHJhbnNwb3J0YWRvcg5Qb3IgTG9jYWxpZGFkZQxQb3IgVmXDrWN1bG8VAwExATIBMxQrAwNnZ2cWAQIBZAIFDw8WAh4HVmlzaWJsZWhkFgoCAQ8PFgIfDmhkFgICAQ8PFgIfBGVkZAIDDxYCHwIFF1JOVFJDIGRvIHRyYW5zcG9ydGFkb3I6ZAIFDw8WAh8EZWRkAgcPFgIfAgUaQ1BGL0NOUEogZG8gdHJhbnNwb3J0YWRvcjpkAgkPDxYCHwRlZGQCBw8PFgIfDmdkFgYCAQ8QDxYGHwwFBnR0ckNvZB8NBRdUdHJUeHREZXNjcmljYW9SZWR1emlkYR8DZ2QQFQMJQXV0w7Rub21vC0Nvb3BlcmF0aXZhB0VtcHJlc2EVAwNUQUMDQ1RDA0VUQxQrAwNnZ2cWAWZkAgMPEGQPFhxmAgECAgIDAgQCBQIGAgcCCAIJAgoCCwIMAg0CDgIPAhACEQISAhMCFAIVAhYCFwIYAhkCGgIbFhwQBQlTZWxlY2lvbmUFATBnEAUCQUMFAkFDZxAFAkFMBQJBTGcQBQJBTQUCQU1nEAUCQVAFAkFQZxAFAkJBBQJCQWcQBQJDRQUCQ0VnEAUCREYFAkRGZxAFAkVTBQJFU2cQBQJHTwUCR09nEAUCTUEFAk1BZxAFAk1HBQJNR2cQBQJNUwUCTVNnEAUCTVQFAk1UZxAFAlBBBQJQQWcQBQJQQgUCUEJnEAUCUEUFAlBFZxAFAlBJBQJQSWcQBQJQUgUCUFJnEAUCUkoFAlJKZxAFAlJOBQJSTmcQBQJSTwUCUk9nEAUCUlIFAlJSZxAFAlJTBQJSU2cQBQJTQwUCU0NnEAUCU0UFAlNFZxAFAlNQBQJTUGcQBQJUTwUCVE9nFgECFGQCBQ8QDxYEHwNnHwhnZBAVqAEJU2VsZWNpb25lBUFDQVJJBEHDh1UOQUZPTlNPIEJFWkVSUkEKw4FHVUEgTk9WQQpBTEVYQU5EUklBDUFMTUlOTyBBRk9OU08RQUxUTyBETyBST0RSSUdVRVMHQU5HSUNPUxBBTlTDlE5JTyBNQVJUSU5TBUFQT0RJDEFSRUlBIEJSQU5DQQRBUkVaDUJBw41BIEZPUk1PU0EIQkFSQcOaTkEJQkFSQ0VMT05BD0JFTlRPIEZFUk5BTkRFUwpCT0EgU0HDmkRFBUJPRMOTCUJPTSBKRVNVUwhCUkVKSU5ITxFDQUnDh0FSQSBETyBOT1JURRhDQUnDh0FSQSBETyBSSU8gRE8gVkVOVE8GQ0FJQ8OTDENBTVBPIEdSQU5ERQ1DQU1QTyBSRURPTkRPDENBTkdVQVJFVEFNQQlDQVJBw5pCQVMUQ0FSTkHDmkJBIERPUyBEQU5UQVMKQ0FSTkFVQkFJUwxDRUFSw4EtTUlSSU0LQ0VSUk8gQ09Sw4EQQ09ST05FTCBFWkVRVUlFTBRDT1JPTkVMIEpPw4NPIFBFU1NPQQdDUlVaRVRBDUNVUlJBSVMgTk9WT1MQRE9VVE9SIFNFVkVSSUFOTwdFTkNBTlRPB0VRVUFET1IPRVNQw41SSVRPIFNBTlRPGEVTUMONUklUTyBTQU5UTyBETyBPRVNURQhFWFRSRU1PWg1GRUxJUEUgR1VFUlJBEEZFUk5BTkRPIFBFRFJPWkEJRkxPUsOCTklBEEZSQU5DSVNDTyBEQU5UQVMORlJVVFVPU08gR09NRVMIR0FMSU5IT1MJR09JQU5JTkhBGkdPVkVSTkFET1IgRElYLVNFUFQgUk9TQURPB0dST1NTT1MIR1VBTUFSw4kNSUVMTU8gTUFSSU5ITwpJUEFOR1VBw4dVB0lQVUVJUkEGSVRBSsOBBUlUQcOaCEpBw4dBTsODCUpBTkRBw41SQQhKQU5EVcONUwRKQVBJEUpBUkRJTSBERSBBTkdJQ09TEkpBUkRJTSBERSBQSVJBTkhBUxFKQVJESU0gRE8gU0VSSUTDkw1KT8ODTyBDw4JNQVJBCkpPw4NPIERJQVMOSk9Tw4kgREEgUEVOSEEISlVDVVJVVFUHSlVOREnDgQxMQUdPQSBEJ0FOVEEPTEFHT0EgREUgUEVEUkFTD0xBR09BIERFIFZFTEhPUwpMQUdPQSBOT1ZBDUxBR09BIFNBTEdBREEFTEFKRVMOTEFKRVMgUElOVEFEQVMJTFVDUsOJQ0lBC0xVw41TIEdPTUVTCE1BQ0HDjUJBBU1BQ0FVC01BSk9SIFNBTEVTEE1BUkNFTElOTyBWSUVJUkEHTUFSVElOUwxNQVhBUkFOR1VBUEUPTUVTU0lBUyBUQVJHSU5PCU1PTlRBTkhBUwxNT05URSBBTEVHUkUUTU9OVEUgREFTIEdBTUVMRUlSQVMITU9TU09Sw5MFTkFUQUwPTsONU0lBIEZMT1JFU1RBCU5PVkEgQ1JVWhZPTEhPLUQnw4FHVUEgRE8gQk9SR0VTC09VUk8gQlJBTkNPB1BBUkFOw4EJUEFSQVpJTkhPCFBBUkVMSEFTClBBUk5BTUlSSU0MUEFTU0EgRSBGSUNBCFBBU1NBR0VNBFBBVFUOUEFVIERPUyBGRVJST1MMUEVEUkEgR1JBTkRFC1BFRFJBIFBSRVRBDVBFRFJPIEFWRUxJTk8LUEVEUk8gVkVMSE8LUEVORMOKTkNJQVMHUElMw5VFUwxQT8OHTyBCUkFOQ08KUE9SVEFMRUdSRQ9QT1JUTyBETyBNQU5HVUUGUFVSRVpBEFJBRkFFTCBGRVJOQU5ERVMOUkFGQUVMIEdPREVJUk8OUklBQ0hPIERBIENSVVoRUklBQ0hPIERFIFNBTlRBTkEJUklBQ0hVRUxPC1JJTyBETyBGT0dPEVJPRE9MRk8gRkVSTkFOREVTC1JVWSBCQVJCT1NBClNBTlRBIENSVVoLU0FOVEEgTUFSSUEQU0FOVEFOQSBETyBNQVRPUxJTQU5UQU5BIERPIFNFUklEw5MOU0FOVE8gQU5Uw5ROSU8TU8ODTyBCRU5UTyBETyBOT1JURRRTw4NPIEJFTlRPIERPIFRSQUlSSQ1Tw4NPIEZFUk5BTkRPF1PDg08gRlJBTkNJU0NPIERPIE9FU1RFGVPDg08gR09Ow4dBTE8gRE8gQU1BUkFOVEUUU8ODTyBKT8ODTyBETyBTQUJVR0kUU8ODTyBKT1PDiSBERSBNSVBJQlUXU8ODTyBKT1PDiSBETyBDQU1QRVNUUkUVU8ODTyBKT1PDiSBETyBTRVJJRMOTC1PDg08gTUlHVUVMFVPDg08gTUlHVUVMIERFIFRPVVJPUxVTw4NPIFBBVUxPIERPIFBPVEVOR0kKU8ODTyBQRURSTwtTw4NPIFJBRkFFTApTw4NPIFRPTcOJDFPDg08gVklDRU5URRZTRU5BRE9SIEVMw5NJIERFIFNPVVpBGFNFTkFET1IgR0VPUkdJTk8gQVZFTElOTwxTRVJSQSBDQUlBREETU0VSUkEgREUgU8ODTyBCRU5UTwxTRVJSQSBETyBNRUwUU0VSUkEgTkVHUkEgRE8gTk9SVEUIU0VSUklOSEETU0VSUklOSEEgRE9TIFBJTlRPUw5TRVZFUklBTk8gTUVMTwtTw41USU8gTk9WTxBUQUJPTEVJUk8gR1JBTkRFBVRBSVBVCFRBTkdBUsOBD1RFTkVOVEUgQU5BTklBUxdURU5FTlRFIExBVVJFTlRJTk8gQ1JVWgVUSUJBVQxUSUJBVSBETyBTVUwWVElNQkHDmkJBIERPUyBCQVRJU1RBUwZUT1VST1MQVFJJVU5GTyBQT1RJR1VBUghVTUFSSVpBTAdVUEFORU1BB1bDgVJaRUEJVkVOSEEtVkVSCVZFUkEgQ1JVWgdWScOHT1NBCVZJTEEgRkxPUhWoAQEwBDcxMjQENzEyNQQ3MTI2BDcxMjcENzEyOAQ3MTI5BDcxMzAENzEzMQQ3MTMyBDcxMzMENzEzNAQ3MTM1BDcxMzYENzEzOAQ3MTM5BDcxNDEENzE0MgQ3MTQzBDcxNDQENzE0NQQ3MTQ2BDcxNDcENzE0OAQ3MTQ5BDcxNTAENzE1MQQ3MTUyBDcxNTMENzE1NAQ3MTU1BDcxNTYENzE1NwQ3MTU4BDcxNjAENzE2MQQ3MTYyBDcxNjMENzE2NAQ3MTY1BDcxNjYENzE2NwQ3MTY4BDcxNjkENzE3MQQ3MTcyBDcxNzMENzE3NAQ3MTc2BDcxNzcENzE3OAQ3MTc5BDcxODAENzE4MgQ3MTg0BDcxODUENzE4NgQ3MTg3BDcxODgENzE4OQQ3MTkwBDcxOTEENzE5MgQ3MTkzBDcxOTQENzE5NQQ3MTk2BDcxOTcENzE5OAQ3MTk5BDcyMDAENzIwMQQ3MjAyBDcyMDMENzIwNAQ3MjA1BDcyMDYENzIwNwQ3MjA4BDcyMDkENzIxMQQ3MjEyBDcyMTMENzIxNQQ3MjE2BDcyMTcENzIxOAQ3MjE5BDcyMjAENzIyMQQ3MjIyBDcyMjMENzIyNAQ3MjI1BDcyMjYENzIyNwQ3MjI4BDcyMjkENzIzMAQ3MjMxBDcyMzIENzIzMwQ3MjM0BDcyMzUENzIzNgQ3MjM3BDcyMzgENzIzOQQ3MjQwBDcyNDEENzI0MgQ3MjQzBDcyNDQENzI0NQQ3MjQ2BDcyNDcENzI0OAQ3MjQ5BDcyNTAENzI1MgQ3MjU0BDcyNTYENzI1OAQ3MjU5BDcyNjAENzI2MgQ3MjYzBDcyNjUENzI2NgQ3MjY4BDcyNjkENzI3MQQ3MjcyBDcyNzMENzI3NAQ3Mjc1BDcyNzYENzI3NwQ3Mjc4BDcyNzkENzI4MAQ3MjgxBDcyODIENzI4MwQ3Mjg1BDcyODYENzI4NwQ3Mjg4BDcyODkENzI5MAQ3MjkxBDcyOTIENzI5MwQ3Mjk0BDcyOTUENzI5NgQ3Mjk3BDcyOTgENzI5OQQ3MzAwBDczMDIENzMwMwQ3MzA0BDczMDUENzMwNgQ3MzA3BDczMDgENzMwORQrA6gBZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnZ2dnFgECWWQCCQ8PFgIfDmhkZAINDw8WAh8OZ2QWFAIBDxYCHw5oZAIDDzwrABEDAA8WBB8DZx4LXyFJdGVtQ291bnQCpgRkAQ8UKwAFFCsABRYGHglEYXRhRmllbGQFHE5vbWVSYXphb1NvY2lhbFRyYW5zcG9ydGFkb3IeCkhlYWRlclRleHQFDVRyYW5zcG9ydGFkb3IeCkh0bWxFbmNvZGVoFgYeD0hvcml6b250YWxBbGlnbgsqKVN5c3RlbS5XZWIuVUkuV2ViQ29udHJvbHMuSG9yaXpvbnRhbEFsaWduAR4NVmVydGljYWxBbGlnbgsqJ1N5c3RlbS5XZWIuVUkuV2ViQ29udHJvbHMuVmVydGljYWxBbGlnbgIeBF8hU0ICgIAMFggeCENzc0NsYXNzBRN0YWJlbGFDb21CYWNrZ3JvdW5kHgtCb3JkZXJTdHlsZQsqJVN5c3RlbS5XZWIuVUkuV2ViQ29udHJvbHMuQm9yZGVyU3R5bGUEHgtCb3JkZXJXaWR0aBsAAAAAAADwPwEAAAAfFQJiZGQUKwAFFgYfEmgfEAUSUk5UUkNUcmFuc3BvcnRhZG9yHxEFBVJOVFJDFgYfEwsrBAIfFAsrBQIfFQKAgAwWCB8WBRN0YWJlbGFDb21CYWNrZ3JvdW5kHxcLKwYEHxgbAAAAAAAA8D8BAAAAHxUCYmRkFCsABRYIHxJoHxAFDERhdGFDYWRhc3Rybx8RBRBDYWRhc3RyYWRvIGRlc2RlHhBEYXRhRm9ybWF0U3RyaW5nBQ57MDpkZC9NTS95eXl5fRYGHxMLKwQCHxQLKwUCHxUCgIAMFggfFgUTdGFiZWxhQ29tQmFja2dyb3VuZB8XCysGBB8YGwAAAAAAAPA/AQAAAB8VAmJkZBQrAAUWCB8SaB8QBRFEYXRhVmFsaWRhZGVSTlRSQx8RBRBEYXRhIGRlIFZhbGlkYWRlHxkFDnswOmRkL01NL3l5eXl9FgYfEwsrBAIfFAsrBQIfFQKAgAwWCB8WBRN0YWJlbGFDb21CYWNrZ3JvdW5kHxcLKwYEHxgbAAAAAAAA8D8BAAAAHxUCYmRkFCsABRYGHxJoHxAFBUF0aXZvHxEFClNpdHVhw6fDo28WBh8TCysEAh8UCysFAh8VAoCADBYIHxYFE3RhYmVsYUNvbUJhY2tncm91bmQfFwsrBgQfGBsAAAAAAADwPwEAAAAfFQJiZGQUKwEFZmZmZmYMFCsAABYCZg9kFhYCAQ9kFgpmDw8WAh8EBSBCcmVubmEgR2FicmllbGxlIFRvcnJlcyBkZSBQYWl2YWRkAgEPDxYCHwQFCTA1MTQzNzY1NWRkAgIPDxYCHwQFCjI2LzExLzIwMThkZAIDDw8WAh8EBQoyNi8xMS8yMDIzZGQCBA8PFgIfBAUFQXRpdm9kZAICD2QWCmYPDxYCHwQFI0JydW5hIENhbWlsYSBTaWx2YSBDbGVtZW50aW5vIEdvbWVzZGQCAQ8PFgIfBAUJMDUwNzQxMzg4ZGQCAg8PFgIfBAUKMjYvMDIvMjAxOGRkAgMPDxYCHwQFCjI2LzAyLzIwMjNkZAIEDw8WAh8EBQVBdGl2b2RkAgMPZBYKZg8PFgIfBAUcQnJ1bm8gQ2V6YXIgQmV6ZXJyYSBkYSBDb3N0YWRkAgEPDxYCHwQFCTA0OTg1MDMyMmRkAgIPDxYCHwQFCjA4LzAyLzIwMTdkZAIDDw8WAh8EBQowOC8wMi8yMDIyZGQCBA8PFgIfBAUFQXRpdm9kZAIED2QWCmYPDxYCHwQFH0JydW5vIEd1c3Rhdm8gUGVyZWlyYSBkZSBDYXN0cm9kZAIBDw8WAh8EBQkwNDY2NTQ2NTVkZAICDw8WAh8EBQoxMC8wNy8yMDEzZGQCAw8PFgIfBAUKMjQvMTEvMjAyMWRkAgQPDxYCHwQFBUF0aXZvZGQCBQ9kFgpmDw8WAh8EBRdCcnVubyBUYWRhc2hpIEtvYmF5YXNoaWRkAgEPDxYCHwQFCTA1MTE0NjY1NWRkAgIPDxYCHwQFCjE3LzA4LzIwMThkZAIDDw8WAh8EBQoxNy8wOC8yMDIzZGQCBA8PFgIfBAUFQXRpdm9kZAIGD2QWCmYPDxYCHwQFKUNhcmxhIENocmlzdGluYSBSb2RyaWd1ZXMgWGF2aWVyIGRhIFJvY2hhZGQCAQ8PFgIfBAUJMDAyOTkzOTM0ZGQCAg8PFgIfBAUKMTQvMDIvMjAwNWRkAgMPDxYCHwQFCjA1LzAyLzIwMjNkZAIEDw8WAh8EBQVBdGl2b2RkAgcPZBYKZg8PFgIfBAUVQ2FybG9zIEFsYmVydG8gRnJvZXNlZGQCAQ8PFgIfBAUJMDQ3NjQ0NDMzZGQCAg8PFgIfBAUKMTAvMDYvMjAxNGRkAgMPDxYCHwQFCjE1LzA5LzIwMjJkZAIEDw8WAh8EBQVBdGl2b2RkAggPZBYKZg8PFgIfBAUYQ2FybG9zIEJhcmJvc2EgZGUgRmFyaWFzZGQCAQ8PFgIfBAUJMDA3NTkzNTM1ZGQCAg8PFgIfBAUKMjAvMTAvMjAwNWRkAgMPDxYCHwQFCjE2LzA4LzIwMjFkZAIEDw8WAh8EBQVBdGl2b2RkAgkPZBYKZg8PFgIfBAUXQ2FybG9zIEVkdWFyZG8gZGEgU2lsdmFkZAIBDw8WAh8EBQkwNTE0OTQwNjZkZAICDw8WAh8EBQoxOC8xMi8yMDE4ZGQCAw8PFgIfBAUKMTgvMTIvMjAyM2RkAgQPDxYCHwQFBUF0aXZvZGQCCg9kFgpmDw8WAh8EBRtDYXJsb3MgTWFnbm8gZGEgU2lsdmEgU291emFkZAIBDw8WAh8EBQkwMTIxMTgwMzNkZAICDw8WAh8EBQowOC8xMi8yMDA5ZGQCAw8PFgIfBAUKMDkvMDgvMjAyMmRkAgQPDxYCHwQFBUF0aXZvZGQCCw8PFgIfDmhkZAIFDxQrAAYPFgIfDmdkAggCCgKmBAIKZxYIAgEPFgIfDmgWAgIBDxYCHwRlZAICDxYCHw5nZAIDDxYCHw5nZAIEDxYCHw8CChYUZg9kFgQCAQ8WAh8EBQVpbmRleGQCAw8PFgQfBAUBMx4PQ29tbWFuZEFyZ3VtZW50BQEzZGQCAQ9kFgQCAQ8WAh8EBQVpbmRleGQCAw8PFgQfBAUBNB8aBQE0ZGQCAg9kFgQCAQ8WAh8EBQVpbmRleGQCAw8PFgQfBAUBNR8aBQE1ZGQCAw9kFgQCAQ8WAh8EBQVpbmRleGQCAw8PFgQfBAUBNh8aBQE2ZGQCBA9kFgQCAQ8WAh8EBQVpbmRleGQCAw8PFgQfBAUBNx8aBQE3ZGQCBQ9kFgQCAQ8WAh8EBQVpbmRleGQCAw8PFgQfBAUBOB8aBQE4FgIeBXN0eWxlBRJmb250LXdlaWdodDpib2xkOztkAgYPZBYEAgEPFgIfBAUFaW5kZXhkAgMPDxYEHwQFATkfGgUBOWRkAgcPZBYEAgEPFgIfBAUFaW5kZXhkAgMPDxYEHwQFAjEwHxoFAjEwZGQCCA9kFgQCAQ8WAh8EBQVpbmRleGQCAw8PFgQfBAUCMTEfGgUCMTFkZAIJD2QWBAIBDxYCHwQFBWluZGV4ZAIDDw8WBB8EBQIxMh8aBQIxMmRkAgcPFgQfAgUXNzEgLSA4MCBkZSA1NTAgaXRlbShucykfDmdkAgkPZBYCAgMPPCsAEQMADxYEHwNnHw9mZAEQFgNmAgECAhYDPCsABQECFg4fFmUfFwsrBgEeCUZvcmVDb2xvcgojHglGb250X0JvbGRnHxgbAAAAAAAAAAABAAAAHglCYWNrQ29sb3IKGx8VAu4QPCsABQECFg4fFmUfFwsrBgEfHAojHx1nHxgbAAAAAAAAAAABAAAAHx4KGx8VAu4QPCsABQECFg4fFmUfFwsrBgEfHAojHx1nHxgbAAAAAAAAAAABAAAAHx4KGx8VAu4QFgNmZmYMFCsAAGQCCw8PFgIfBGVkZAINDw8WAh8EZWRkAg8PZBYEAgMPPCsAEQMADxYEHwNnHw9mZAEQFgVmAgECAgIDAgQWBTwrAAUBAhYIHxYFE3RhYmVsYUNvbUJhY2tncm91bmQfFwsrBgQfGBsAAAAAAADwPwEAAAAfFQJiPCsABQECFggfFgUTdGFiZWxhQ29tQmFja2dyb3VuZB8XCysGBB8YGwAAAAAAAPA/AQAAAB8VAmI8KwAFAQIWCB8WBRN0YWJlbGFDb21CYWNrZ3JvdW5kHxcLKwYEHxgbAAAAAAAA8D8BAAAAHxUCYjwrAAUBAhYIHxYFE3RhYmVsYUNvbUJhY2tncm91bmQfFwsrBgQfGBsAAAAAAADwPwEAAAAfFQJiPCsABQECFggfFgUTdGFiZWxhQ29tQmFja2dyb3VuZB8XCysGBB8YGwAAAAAAAPA/AQAAAB8VAmIWBWZmZmZmDBQrAABkAgUPFCsABmQCAQIKZgIUZxYMAgEPZBYCAgEPFgIfBAUGMSBkZSAwZAICDxYCHw5oZAIDDxYCHw5oZAIEDxYCHw8CARYCZg9kFgQCAQ8WAh8EBRBpbmRleCBmaXJzdGluZGV4ZAIDDw8WBB8EBQExHxoFATEWAh8bBRJmb250LXdlaWdodDpib2xkOztkAgUPFgIfDmhkAgYPFgIfDmhkAhEPDxYCHw5oZGQCEw8PFgIfDmhkZBgDBR5jdGwwMCRDb3JwbyRndlBvbnRvQXRlbmRpbWVudG8PPCsADAEIZmQFFWN0bDAwJENvcnBvJGd2VmVpY3Vsbw88KwAMAQhmZAUfY3RsMDAkQ29ycG8kZ3ZSZXN1bHRhZG9QZXNxdWlzYQ88KwAMAgICBwgCN2SUmH1QczNxQXdXJhTxY8AY4IBeoA==',
      '__VIEWSTATEGENERATOR': '197C39C5',
      '__EVENTVALIDATION': '/wEdAOIBEKGGFNi9DeFtV2JeIfwGeSXO/6Q5tQwH9oSQ2H3IPRtOaN/oFGq3DBGvHyrcLTX50EvyWW/iBMUVLgXGC+/7d9SeC1v3+0thGNhXTkRqnmkAJ0mzcT4wkAVgKO2t9ncaDvsBAv6M01jLiaXzYtQsT4fEXfFET2ukvMd7DHtWQcU6HcgYtHX0knQEOC3ZnR5w5ZOo9Kd54Z1+cKzaGQLdulXS4qijvASlyD2CMYP2KkYhYFzTMnLekmdbJcBipr/HAUYWFaM/s7S7GbhtPz74vBwH4mx4LIbMziJtyHyPRR4pNarJBKWSPdab/Yrdl+cg13oeixBe8HdwSLQCAiUDobK1pzs+UhfuCfigS4iLQodfFMtx1CZVjVfg64O5pFqy+V+iCuz6n4oRleoSunwRB465De0qU0GcR1/93qThxEKBFaQjAXswm7LUOp4vwuPWQUYAAdWcOOZPw1HTKead2UWpaiEXTFaLktKHzIgLSLKdtc/N4kq8AQgnlVcLzoLf/j3U6ku1O6ql8+QmdeXpJjyErd7d7lPvUcJ+nd0PfQplzdxUYbfL8sef2KnVn/Gc7XOtyDLnHW7xeyZOlfdEZomArnQvSewc3UxQTFoBeR/X2OwZ6NLdox6NsGvjGX4TgcFbhU/If3+9tjwkWhgjrRswCocs/w3kqojgUtBt4mmsJpQ2RBKq7mQXZHBrROEulN/ELEWZnGph95KkfB/MeOczQkzgy7oxmkWU1CSoXRWyNfqB1+84ryXlcoCEoSJthGQZ5r1PVg6Jtj4aVHVKRpfPc25QcKB8bd2QXF1JkEMmubs5WBDKI2BIh1svyCdvH2TM+bhXkHmkiiztYPlKyTKyWHNnUlKI6DT69sE99Hp6IyZzsl3bCtAeGKcD4bkcm7XuXNQUL+eV+e+6IEGc2L63Mq8G+d5X4m4Kmfbf/NkIuzxGAoPqe9gqwBWYOSdJ2k7DBtPn6uAfh5fB9f87Tf/PgBzqe9+zGVvsRr5elS9M92sa+f8O1jll2upOuA64VRjYRCFM1RT6a9r4rRovvfBsvt1jBChNGvtDQ4rFa/SRnucVUbokTCq9eD0xjrUQmgPYDMZM4itw/J7KX7yuLCU1nShL9djK6e05uPHGrERPRpe81npJkr35UMKUdcO/lodWzedNtJjWzLp7xj07hDMw4oG1TB/CpdzfoGjQ8u6bpDw44PemTIxpAPFY+0+2R9837HghwI2w2YW4sR8jJxUjAxpJrnn1LwUSwNVrYInfxSXh9ioDjMPAJUCh+vDkGc+ihsTFQd4qUFKpDlYRgYNIz+q8CcVnsVMVjtJxmOrWglpLdDOjp9xDszzrEYOhTKAU7Nj7+6DC9JgWwLd+IOWY0w+0thLIvV+L8Taje6G2zaK3HhbmDfXPMVyEoYiOmycBqDH9I86Ylp2KZ4uBBcJ2UtigSIhO61cA7yZKmeZe5a2ooNgLj7QJDy7tB9Ch5xGVb1a9MhEzhq7mqwm6uHfuISjDJQo8TTiAzNEuadvFDSvywNDdazqoLpphNlGyOXUYQxJJR1zqs2sZ5p4HF/cNBY3f41L0320wkpmoqf61d9F3NYjwcm3aWVLRJnAZv4stKfeQ2U65fA6pUZ9SUsq1ppN2/2tT2GRJewzqPnysqsuBID9yCk68lQtHQDNxc832KKNURiNU51ukKqUBoz6FwvQJGYj3i0DkMoLPnCKLg1ti4QEe2jEdezJQaAn2qVnu/rlNPexygBS3Irj4psQLInMe5JNMdzsgwDDl23NL+w1XI+Wrh8l1VTixPTZY6Hzp4dNQ0/Kt4gSlQhhclJB7nNazeIEGoj9yJFZ9UkRkbfTPO6CwD+pnfq3jnq2fNPwbK1tFS7nBSjKnVI4XKtbWrO8UrL7xtqo7erKnL16C2Jq3s/2fblhUrO4Rz4X7MNibxxkXR7hitywOFRZFEIUd85ixeyP69ZkaMAy64M90wAH5ZH7fyxbOkG3Ir6aC7HNllIS/kFHpxMgKcRc/SAsamz1qX0IUeKO2Ob0CvEeDOi5RYjCEVi4iuHowXfKUPVJ9dNuVsCx0Dnn929qfRVK+hbjwpqme61Rg0rIgS1TY1n5saBSDvcr/JlT2nMtgmM1U937NPzkbFAMQIgKWAZUQ36Vt44iSKpUswmrx+9ECk18XSksGw0JOdHEhodNE3jm5dIdU3fH8DtLf+IEWj/Y0VP2Ld1NRiwjjUDPDtydR7S4IvE7NBmQfBD5A87fFg0/kaV9zr8ISeJ+V5+DG7vXFaOWaH22c1ERdjJZyIYYHc4oSZA7ZKyhrzieX+ZsDwQfbspsJ7159xwDAObPfZcG4l06YaB5fjjvtVdaib8P5ftnsgicWbUjn+mK4swluTeDQnXeI9S2WRrCJREM1TCwUN1/DUhVBI/sWvcmrfh38RBGKYh8K34Z2Mddh33XjCzREsnnEVuR4ScIDIfZfP06ms8gfLMgOONggqPAb0AnZGKqDoB+ntVMYgaU2/niDdVNcBFMOKbHjpILFUFryLSWtbUMZPTP2Do8sYX4HvGd79OhnVCQnOQsYX5ROZSJtM29aOr06IkJf5ggw6CGUuDvM1XSuawOQONvz+CopVYu2aXYcZotT7R4j7xE7u3zQjhbtNtPh3FUk42NvFbrkezAW2OcGY4jY4fKIsH07CQ96HSjp9fUXjyOKUH0/KwRujLSeqbXLaH0DblwWuvLAEmtrd55nrD7waHx0Ndv9QHhorWTsWm3IpxwJg9XvDBNaitwojP01HACTMm5Y6MWxvokff3FviPnr+wk6e3HgdY2zeO/V/C8pJd1XcmqZyBr/GDX+auYUBdPl/6CJ1xVTa0uf5bn60ZGPwMX2AOOg8uq4SRUzi+iB+08gxqVaD3jNypVy01wZ+O2dw2tyUkcFI/mArvQxpIkW/Xlw1PE06aheEVTmXutYiMktX6sDuDc2VocHZHPD5vqm37szc/yaykkn9ej7WYBhTGqtCJz58wlRK2Eije+Ss3H2BC7qDjddFothwbV2V1YFwgJQ776uPxu5EJNbaCPWmtxsLsYqvv+kuLYKayKddfhvPKxqTeq3/vj+R4LSZSfgSgkxxzPNGYPgdk+Ih5oHZzHp43aK/JjlSTnzg+/X9fbKyFXrTYcZR9JUYbmqR3NApKYVwe3uOsAOV3tWDPzfKVZ9PVcB0aDqu1ZO2zM5IZZQznb9rqNUdB31Osf5gOVhALKzZ97ZcaBErgBQo+ovMnw1BkfWnV7k5VtAHdGnXxLrXDBG7JYy2xwhC7KdclCp1SGDNjxTdstz/eBESOwWRStVf71fAMRIf2WNvxdOAE0mSYJCBI/cU3JXu857ieZykGHeKnbLLiFCJvwIY5LYzjV7hwIsTUZf8fd9VCu2gKenn1rIrLA6SjVICyt1eKP+XKi1jR7Am96z4z4VfB20LK5LS6XJjw1E1sUsprRhyNn54Q073tLiSFSNANkMo5NthAtHpzpk+xMliUmIDL+n0ZnhL9Ev3nSTHAKAVkFPNjaNbdX8qWgDYqFzSlzBbJfKNGWMHHe2+gwaSSPJ4nBw83tbWwB21G3GimF4Cph0a8EnBpxs34kcWmcdX3F7mlbIE4zC1Sml0zMfFtJrvkXaM9ljh+FOvEH0XjBoJlfsaQFtI0iGue1hv8tHsZaH66o7lfRZzANevQX73OisXDJD/iEsc6UR0/fBMrgjH6dmUsi9ZYY0hmGCI3JN+wzjaxbLQnfy6lh8bgGVUwnPlsk0fNfnSN2rBiIKjkPeIhpg1poZlld8TqrrXH8VX0Xuy9WH0f1+N2nL4X6q8cHkk9wdHCu8k6ek4I+8Rlm3Dw7PesMvQwQBaKzizSfUCA6whrGZAO13flkX201zTBlhQBGfAwUXsgjQ7mFpiLqvI2aTCYfKU3qKf+T9pXGWm13IFzyguDaNwSeyfnQO6vbD9kEoF5a1BCgFnF5VvBAJL1O3MtB0aLAKceIlWvjggvtompVVUKKL1A82qmojwYIXOlYhnhZqzedDeb2o76fPi6BTJPOglMWSVPny2yMbiqRtndw9xUXDgadyOOQek1riCgqEYP1xbBe9tTNfVW0kNNTevC3+aZLACKRaXfx1n/P4sOojb7Ho17kppYNpPr8NwrIFgiaLVT0cS4NL1BMz4Zw7nWijgkELudF0cGs7ejQHHhkKCa+Yyjo05nmHzZnt8GZ6JV3ZdPscHFCZ5kmRAxkmKRO+crugPHqw8QLevVoG8vC2oqMVxmMu8ewxpYCh0JMLRPq/scucet70CjZCToXewlN1mvnidec9RfC7rF9gdw7bZaIsxtrQ0+eZWdM49g3xubj82tSlCrcmDj43iai4RRtvFufsZI6HpRiSTv85XbI6jlsdOKbEVlKPn1KpebuZtKhAGjydloPfPLXHimwuwsuC2FXJ7uH/AoHUy5DJP/kl6JPQDCp7lGzPvYEoD3rpVXUtGhD6U0cgLg6imio/RX82lM1CUzjM41X/kJaswrBy+shWet3Vqo6mm3RD6vg7y0ofuUe7z32HsmUDdwjS/jkA5vS15Wo9PCdYklTMkkdO0qA7RDxCon60yBMFPc/u1SyVRn1amGF/hm/gSd/JECexR9clUkitZEDe2OHT7DC+xBVbcv/5tYXz3Ku/gLdY7IRL+sPBNc0OgQ94+boeisePl+ehoo3BAC8r8UowZUqpkoF0uPDIh7QGp8+8r7o+dn7sQEfhQbepsgPKzD7NK9CjiOihxfqvGzQVy+dUTefaT/SfJs8Wh2uRcoXGU23e3q5d6A/6Z2CV6Bq7AV/QXvE4TvhrnsivBTs9ZaILHtWcpu4ibfMWD+7PnzcPYVbolcZEAYjMjkSVZ3JN',
      'ctl00$bMostraAlerta': 'true',
      'ctl00$Corpo$hfPnlConsulta': '1',
      'ctl00$Corpo$rbTipoConsulta': '2',
      'ctl00$Corpo$ddlTipo': 'TAC',
      'ctl00$Corpo$ddlUF': 'RN',
      'ctl00$Corpo$ddlMunicipio': '7221',
      'g-recaptcha-response': ''
    }

    response = requests.post('https://consultapublica.antt.gov.br/Site/ConsultaRNTRC.aspx/consultapublica', headers=headers, cookies=cookies, data=data)
    return response

In [3]:
def generate_page_numbers(number):
    if number==0:
        return ['0']
    
    elif number<11:
        return generate_page_numbers(0) + [str(number).zfill(2) for number in range(1,10)]
    
    else:
        return generate_page_numbers(10) + list(map(str, range(10,number)))


In [4]:
def scraper(n):
    
    d = {'Transportador': [], 'RNTRC': [], 'Cadastrado desde': [],
         'Data de Validade': [], 'Situação': []}
    
    page_numbers = generate_page_numbers(n)

    for number in page_numbers:
        html = requester(number)
        soup = BeautifulSoup(html.text)
        table = soup.find("table", {"id": 'Corpo_gvResultadoPesquisa'})

        trs = table.find_all('tr')

        for tr in trs:
            tds = tr.find_all('td')
            for key,td in zip(d,tds):
                d[key].append(td.text)
    
    return d


In [5]:
d = scraper(5)

In [6]:
df = pd.DataFrame(d)
df

,Transportador,RNTRC,Cadastrado desde,Data de Validade,Situação
0,Brenna Gabrielle Torres de Paiva,051437655,26/11/2018,26/11/2023,Ativo
1,Bruna Camila Silva Clementino Gomes,050741388,26/02/2018,26/02/2023,Ativo
2,Bruno Cezar Bezerra da Costa,049850322,08/02/2017,08/02/2022,Ativo
3,Bruno Gustavo Pereira de Castro,046654655,10/07/2013,24/11/2021,Ativo
4,Bruno Tadashi Kobayashi,051146655,17/08/2018,17/08/2023,Ativo
5,Carla Christina Rodrigues Xavier da Rocha,002993934,14/02/2005,05/02/2023,Ativo
6,Carlos Alberto Froese,047644433,10/06/2014,15/09/2022,Ativo
7,Carlos Barbosa de Farias,007593535,20/10/2005,16/08/2021,Ativo
8,Carlos Eduardo da Silva,051494066,18/12/2018,18/12/2023,Ativo
9,Carlos Magno da Silva Souza,012118033,08/12/2009,09/08/2022,Ativo
